# Inizializzazione università e SCA

In [1]:
from SmartContractAuthority.sca import SmartContractAuthority
from University.university import University
from Student.student import Student
import json
from SimulationUtils.simulationUtils import *

SCA_PATH = './smart_contract_authority/'
UNIVERSITIES_PATH = './universities/'
STUDENTS_PATH = './students/'
CONTRACTS_BUILD_PATH = './SmartContracts/build'

## Creazione Smart Contract Authority
Viene associato un account eth alla Smart Contract Authority che fa il deploy dei propri contratti

In [2]:
SmartContractAuthority.create_sca(SCA_PATH, CONTRACTS_BUILD_PATH)

SmartContractAuthority deployed at address: 0x14e32DA25bA65D0eE86B14E808E404F03b83DaAf
SCA creata con successo e configurazione salvata.


### Creazione dell'istanza
Viene effettivamente caricata l'istanza della SCA a partire dal file JSON di persistenza generato nel blocco precedente

In [3]:
sca = SmartContractAuthority(SCA_PATH+"SCA", CONTRACTS_BUILD_PATH)

## Creazione Università
In questo blocco vengono create le università della rete assegnando loro un nome, che poi risulterà nelle directory dedicate, ed un account eth tramite il quale faranno il deploy dei contratti per la gestione di credenziali e studenti.

In [4]:
University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

SIDSmartContract deployed at: 0x870FD6C3A4DF6dBA76a1B66D7FeDfFA4e3d4921e
CIDSmartContract deployed at: 0x4B05bb3Dd86752F35775f8c765cef48e3C5fa39D
Università creata con successo.
SIDSmartContract deployed at: 0xD4fAAF672999feE20CA764B730CeFae331ec50Ae
CIDSmartContract deployed at: 0x841fd76697cce5e5017653a152A3d658B20e42b0
Università creata con successo.


### Creazione dell'istanza
Viene effettivamente caricata l'istanza delle università a partire dal file JSON di persistenza generato nel blocco precedente

In [5]:
uni_origin = University(UNIVERSITIES_PATH+'University_UNISA', CONTRACTS_BUILD_PATH)

uni_erasmus = University(UNIVERSITIES_PATH+'University_ENSICAEN', CONTRACTS_BUILD_PATH)

## Creazione studente

In [6]:
Student.create_student(STUDENTS_PATH)


Studente creato e salvato in: ./students/Student_AntonioCarbone\persistency\student_data.json


In [25]:
studente = Student(STUDENTS_PATH+'Student_AntonioCarbone')

Studente 'Antonio Carbone' caricato da ./students/Student_AntonioCarbone


## Accreditamento università create

In [8]:
UID, sca_contract_address = sca.register_university(uni_origin)
uni_origin.update_uid(UID)
uni_origin.update_sca_contract_address(sca_contract_address)

UID, sca_contract_address = sca.register_university(uni_erasmus)
uni_erasmus.update_uid(UID)
uni_erasmus.update_sca_contract_address(sca_contract_address)

Registrazione Università (UID: unisa) completata con successo. Transaction Hash: bb79e65c844edc07543aa3631436f9af09ec151bab62ae2d49d700fbe8b94621
Registrazione Università (UID: ensicaen) completata con successo. Transaction Hash: a4fd7d7d1a6705676cccaec6bbbf5315349ec64a1d8ff9f69aa064222483e332


## Immatricolazione presso università di origine

In [9]:
credenziale_studente,sid = uni_origin.register_student(studente)
studente.set_sid(sid)
studente.save_credential(credenziale_studente)

Registrazione SID completata con successo. Transaction Hash: 76a27c3c4a1cb9233177aa6601cb83cbd9cf7d6f5426ec4928442fcb139972f0
Registrazione CID completata con successo. Transaction Hash: b076a5192ec61643796f2560df7e608939919b92b3f4c1784ed4c6a4c9873748
Credenziale salvata in: ./students/Student_AntonioCarbone\persistency\credentials\credential_0.json


## Immatricolazione presso università estera

In [10]:
uni_erasmus.register_erasmus_student(studente, studente.credentials[0].toJSON())
uni_erasmus.update_erasmus_students(studente, credenziale_studente.CID)

Il CID associato alla credenziale fornita risulta valido.
Firma dell'issuer verificata correttamente.
Il SID contenuto nella credenziale è valido.
Firma dello studente verificata correttamente.
Complimenti Antonio Carbone 
Immatricolazione Erasmus completata!
ENSICAEN ti dà il benvenuto!


## Richiesta e rilascio credenziale carriera

In [11]:
career_credential = uni_erasmus.request_career_credential(studente)
studente.save_credential(career_credential)

Credenziale di immatricolazione dello studente valida.
Il SID fornito è valido.
Challenge di autenticazione superata.
Registrazione CID completata con successo. Transaction Hash: e9b40201512ea4a06bf1b225dfb95c3ea5ceb4b1b2e17b1af01db2992d5995c7
Complimenti Antonio Carbone 
Richiesta della tua credenziale carriera completata!
Arrivederci da ENSICAEN!
Credenziale salvata in: ./students/Student_AntonioCarbone\persistency\credentials\credential_1.json


## Costruzione della credenziale parziale

In [12]:
partial_credential = studente.build_shared_credential()

studente.save_credential(partial_credential)


Seleziona una credenziale da condividere:
0: CID = CID:unisa:1
1: CID = CID:ensicaen:1

Seleziona le proprietà da condividere (y/n):

Credenziale parziale costruita con successo:
{
    "certificateId": "CID:ensicaen:1",
    "studentId": "SID:unisa:1",
    "universityId": "UID:ensicaen",
    "issuanceDate": "2023-10-01",
    "properties": [
        {
            "typology": "ErasmusInfo",
            "data": {
                "programName": "Erasmus+ Europe",
                "startActivity": "2024-09-01",
                "endActivity": "2025-01-31"
            },
            "nonce": "17529447317422497",
            "merkle_proof": [
                [
                    "2e7832730b37b90ad152a014457793a5e77d74312726b065313a67f5eaba6378",
                    "R"
                ],
                [
                    "d6478f270f85ef93e81ca699e3acc5c001bc5078133577de799fd06dd02c5320",
                    "R"
                ],
                [
                    "5b3479e7a7d0665cde575

## Condivisione credenziale

In [13]:
uni_origin.validate_shared_credential(studente,partial_credential.toJSON())

L'università che ha rilasciato la credenziale condivisa è fidata.
Il SID presente nella credenziale è valido.
Challenge di autenticazione superata.
Le property sono state verificate correttamente!
La credenziale condivisa è valida


## Revoca credenziale di immatricolazione

In [ ]:
uni_origin.revoke_cid("CID:UNISA:1")

## Revoca credenziale carriera

In [ ]:
uni_erasmus.revoke_cid("CID:ENSICAEN:0")

## Manomissione di una credenziale
In questo blocco, tramite l'ausilio di una funzione che manomette il JSON di una credenziale, viene testata la resistenza della soluzione al threat model 1.3.2 e, implicitamente, anche al threat 1.3.3.
Difatti, la credenziale modificata, genera una condizione di eccezione poichè viene verificata una incongruenza nelle proof di merkle (si tratta di una credenziale parziale)
Nel caso di una credenziale totale, invece, la situazione è del tutto simile, come possiamo osservare nel secondo snippet di codice. Viene infatti rilevata l'invalidità della firma.

In [14]:
parsed_partial_credential = json.loads(partial_credential.toJSON())
tampered_credential = tamper_all_credential_properties(parsed_partial_credential)
uni_origin.validate_shared_credential(studente, json.dumps(tampered_credential, ensure_ascii=False, indent=4))

L'università che ha rilasciato la credenziale condivisa è fidata.
Il SID presente nella credenziale è valido.
Challenge di autenticazione superata.


Exception: Errore durante la verifica della firma dell'issuer: Inconsistenza rilevata nella merkle proof della property 1 (Tipo: Residence). Root calcolata: 5d24c28f48cc287a39de1842604c6b9d5e39c7c0d3f50ad57d825d6543397f52, Root attesa: 2b857145324b4ded3862cb6e8c2c29b648d88a65578193eaf46cf7d7cea2bb74

In [27]:
print(studente.credentials[1].toJSON())

{
    "certificateId": "CID:ensicaen:1",
    "studentId": "SID:unisa:1",
    "universityId": "UID:ensicaen",
    "issuanceDate": "2023-10-01",
    "properties": [
        {
            "typology": "ErasmusInfo",
            "data": {
                "programName": "Erasmus+ Europe",
                "startActivity": "2024-09-01",
                "endActivity": "2025-01-31"
            },
            "nonce": "17529447317422497"
        },
        {
            "typology": "Residence",
            "data": {
                "typology": "off-campus",
                "address": "Via Università 14, City"
            },
            "nonce": "17529447317421959"
        },
        {
            "typology": "Scholarship",
            "data": {
                "amount": 1002,
                "unit": "EUR",
                "payments": 1
            },
            "nonce": "17529447317421687"
        },
        {
            "typology": "Course",
            "data": {
                "name": "Cyber

In [28]:
parsed_partial_credential = json.loads(studente.credentials[1].toJSON()) #la credenziale 1 nella demo è quella che lo studente ha ottenuto al termine della carriera ERASMUS
tampered_credential = tamper_all_credential_properties(parsed_partial_credential)
uni_origin.validate_shared_credential(studente, json.dumps(tampered_credential, ensure_ascii=False, indent=4))

L'università che ha rilasciato la credenziale condivisa è fidata.
Il SID presente nella credenziale è valido.
Challenge di autenticazione superata.


Exception: Errore durante la verifica della firma dell'issuer: 

## Ente certificatore fasullo

## Ente certificatore malintenzionato

## Sosia
Per simulare lo scenario di un sosia (Threat 1.3.6), proviamo a far condividere la credenziale dello studente dei precedenti esperimenti ad un sosia, immatricolando quest'ultimo alla stessa università, anche se questo non influenza l'esito dell'esperimento.
Lo studente difatti, non riesce a superare la challenge di autenticazione, e viene pertanto generata una eccezione sul controllo della firma del nonce.

In [29]:
Student.create_student(STUDENTS_PATH)
studente_sosia = Student(STUDENTS_PATH+'Student_SosiaSosia')
credenziale_studente_sosia,sid_sosia = uni_origin.register_student(studente)
studente_sosia.set_sid(sid_sosia)
studente_sosia.save_credential(credenziale_studente_sosia)


Studente creato e salvato in: ./students/Student_SosiaSosia\persistency\student_data.json
Studente 'Sosia Sosia' caricato da ./students/Student_SosiaSosia
Registrazione SID completata con successo. Transaction Hash: 870722aa4d01b9d17b84541294182f9289108d4ad94c07cff373ec42cb006e67
Registrazione CID completata con successo. Transaction Hash: 0ac10231343b1f9e295f847359060cbd520a33ee6e303fbefbf600e52ac8be96
Credenziale salvata in: ./students/Student_SosiaSosia\persistency\credentials\credential_0.json


In [30]:
uni_origin.validate_shared_credential(studente_sosia,studente.credentials[1].toJSON())

L'università che ha rilasciato la credenziale condivisa è fidata.
Il SID presente nella credenziale è valido.


Exception: Errore durante la verifica della firma dello studente: 

## Forgiatore di credenziali